In [1]:
import xml.etree.cElementTree as ET
import pprint
import re
import gc
import csv
import codecs 
import sqlite3
import pandas as pd
import os
from humanize import naturalsize
import requests

# Declare Variables


OSM_FILE = "Phoenix.osm" # Total set of about 34M
SAMPLE_FILE = "phxsmall.osm" # small subset about  3.4K
TEST_FILE="phxmed.osm" #intermediate subset about 340K 
     # get osm data for project
url = "http://overpass-api.de/api/interpreter?data=%28node%2832%2E6020062%2C%2D112%2E9699853%2C34%2E318%2C%2D111%2E036001%29%3B%3C%3B%29%3Bout%20meta%3B%0A"
response = requests.get(url)

with open('Phoenix.osm', 'wb') as file:
    file.write(response.content)


street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons","Way", "Circle", "Key","Terrace", "Garden"]
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Ave.":"Avenue",
            "Rd.": "Road",
            "BLVD": "Boulevard",
            "Dr.": "Drive",
            "PL": "Place",
            "Pl": "Place",
            "Ln":"Lane",
            "Ct": "Court",
            "Blvd": "Boulevard",
            "Cir": "Circle",
            "Dr": "Drive"
          }

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


                    

            

In [2]:
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()})

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

def audit_address(filename):
    expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", "Trail", "Parkway", "Commons", "Way", "Circle", "Key", "Terrace", "Garden"]
    street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
    postcode_key = 'addr:postcode'
    street_key = 'addr:postcode'
    state_key = 'addr:state'
    city_key = 'addr:state'
    county_key = 'addr:county'
    
    postcode_types = {}
    street_types = {}
    state_types = {}
    city_types = {}
    county_types = {}
    
    for event, elem in ET.iterparse(filename, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k']== postcode_key:
                    if not re.search(r"^8[56]\d{3}-?(\d{4})?$", tag.attrib['v']):
                        if tag.attrib['v'] in postcode_types:
                            postcode_types[tag.attrib['v']]+=1
                        else:
                            postcode_types[tag.attrib['v']]=1
                elif tag.attrib['k']== street_key:
                    m = street_type_re.search(tag.attrib['v'])
                    if m:
                        street_type = m.group()
                        if street_type not in expected:
                            street_types[street_type]=street_types.get(street_type,0)+1
                elif tag.attrib['k']==state_key:
                    state_types[tag.attrib['v']]=state_types.get(tag.attrib['v'],0)+1
                elif tag.attrib['k']==city_key:
                    city_types[tag.attrib['v']]=city_types.get(tag.attrib['v'],0)+1
                elif tag.attrib['k']==county_key:
                    county_types[tag.attrib['v']]=county_types.get(tag.attrib['v'],0)+1
                    
    print '\nPostcodes out of norm:\n', postcode_types, '\nStreets out of norm:\n', street_types, '\nStates:\n', state_types, '\nCities:\n', city_types, '\nCounties:\n', county_types
            
            
def update_name(name, mapping):
    street=street_type_re.search(name).group()

    name=name.replace(street, mapping[street])

    return name


#clean_element function take tag['value'] and tag['key'] as input and return the updated tag values 
def clean_element(tag_value, tag_key):
    
    ## clean postcode 
    if tag_key=='postcode':
        if tag_value[0:2]!='85' or len(tag_value)!=5:
            ## find postcode start with 'AZ' and remove the 'AZ' 
            if tag_value[0:2]=='AZ': 
                    tag_value=tag_value[-5:]
     
                    #print (tag_value)
                            
            ##  find cases that using full address as postcode and extract the postcode using re module
            else:
                if len(tag_value)>5:
                    #print(tag.attrib['v'])
                    pc=re.search('(85\d{3})', tag_value)
                    if pc:
                        tag_value=pc.group()
              
    ## clean state name, use uniformat 'AZ'       
    elif tag_key=='state':
        tag_value='AZ'
        
    ## clean street suffix, change abbrivations to full street suffix        
    elif tag_key=='street':
        street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
        full_addr=tag_value
        m = street_type_re.search(full_addr)
        if m:
            street_type = m.group() 
            if street_type not in expected:
                if street_type in mapping:
                    tag_value=update_name(full_addr, mapping)
    return tag_value
                             
## Clean and shape node or way XML element to Python dict

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
   
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
   
    ## clean node element
    if element.tag=='node':
        for primary in element.iter():
            for i in node_attr_fields: 
                if i in primary.attrib: 
                    node_attribs[i]=primary.attrib[i]
        if len(element)!=0:
            for j in range(0, len(element)): 
                childelem=element[j]
                tag={}
                if not problem_chars.search(childelem.attrib['k']): ## ignor problematic element
                    tag["id"]=element.attrib["id"]
                    tag["type"]=default_tag_type
                    tag['value']=childelem.attrib['v']
                    if ":" in childelem.attrib['k']:
                        k_and_v=childelem.attrib['k'].split(':',1)
                        tag["type"]=k_and_v[0]
                        tag["key"]=k_and_v[1]
                        if tag["type"]=='addr':
                            tag["value"]=clean_element(tag["value"],tag["key"]) ## call clean_element function
                    else:
                        tag["key"]=childelem.attrib['k']
                        if tag["type"]=='addr':
                            print(tag_value, tag["key"])
                            tag["value"]=clean_element(tag["value"],tag["key"])
                tags.append(tag)
                
        return ({'node': node_attribs, 'node_tags': tags})            
                    
    ## handle way element               
    elif element.tag=='way':
        for primary in element.iter():
            for i in way_attr_fields: 
                if i in primary.attrib: 
                    way_attribs[i]=primary.attrib[i]   
        
        if len(element)!=0: 
            for j in range(0, len(element)): 
                childelem=element[j]
                tag={}
                if childelem.tag=='tag':
                    if not problem_chars.search(childelem.attrib['k']):
                        tag["id"]=element.attrib["id"]
                        tag["type"]=default_tag_type
                        tag["value"]=childelem.attrib['v']
                        if ":" in childelem.attrib['k']:
                            k_and_v=childelem.attrib['k'].split(':',1)
                            tag["key"]=k_and_v[1]
                            tag["type"]=k_and_v[0]
                            if tag["type"]=='addr':
                                tag["value"]=clean_element(tag["value"],tag["key"]) #call clean_element function
                        else:
                            tag["key"]=childelem.attrib['k']
                            if tag["type"]=='addr':
                                tag["value"]=clean_element(tag["value"],tag["key"]) #update tag values
                    tags.append(tag)
                    
                elif childelem.tag=='nd':
                    #print (childelem.attrib['ref'])
                    way_node={}
                    way_node['id']=element.attrib['id'] 
                    way_node['node_id']=childelem.attrib['ref']
                    way_node['position']=j
                    #print(way_node)
                    way_nodes.append(way_node)
                    
        return ({'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags})
    

## process the file, clean and write XML into csv

def process_map(file_in):
    with codecs.open(NODES_PATH, 'wb') as nodes_file, codecs.open(NODE_TAGS_PATH, 'wb') as nodes_tags_file, codecs.open(WAYS_PATH, 'wb') as ways_file, codecs.open(WAY_NODES_PATH, 'wb') as way_nodes_file, codecs.open(WAY_TAGS_PATH, 'wb') as way_tags_file:
                
        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)
        
        nodes_writer.writeheader()
        
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()
       
        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])
                    

def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()         
            
## create samples of original osm
def sample_data (large, small, medium): 
    
    k = 10000 # Parameter: take every k-th top level element, take small sample
    m = 100 # take intermediate sample  
    with open(small, 'wb') as output1, open(medium,'wb') as output2:
        output1.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        output1.write('<osm>\n  ')
        output2.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        output2.write('<osm>\n  ')
    # Write every kth and mth top level element
        for i, element in enumerate(get_element(large)):
            if i % k == 0:
                output1.write(str(ET.tostring(element, encoding='utf-8')))
                output2.write(str(ET.tostring(element, encoding='utf-8')))
            elif i % m == 0:
                output2.write(str(ET.tostring(element, encoding='utf-8')))
        output1.write('</osm>')
        output2.write('</osm>')

In [3]:

sample_data(OSM_FILE, SAMPLE_FILE,TEST_FILE)
# audit_address(SAMPLE_FILE)
# audit_address(TEST_FILE)
# audit_address(OSM_FILE)
process_map(OSM_FILE)

In [4]:
#Create database from text files
if os.path.isfile('phoenix.db'):
    os.remove('phoenix.db')
conn=sqlite3.connect('phoenix.db')
conn.text_factory = lambda x: unicode(x, 'utf-8', 'ignore') #added to fix an error showing extra spaces and then not being able to laod into the db
cur = conn.cursor() 
cur.execute("CREATE TABLE nodes ( id INTEGER PRIMARY KEY NOT NULL, lat REAL, lon REAL, user TEXT, uid INTEGER, version INTEGER, changeset INTEGER, timestamp TEXT )")
conn.commit()
node_df = pd.read_csv('nodes.csv', dtype=object)
node_df.to_sql('nodes', conn, if_exists='append', index=False)


cur.execute("CREATE TABLE nodes_tags (id INTEGER,\
    key TEXT,\
    value TEXT,\
    type TEXT,\
    FOREIGN KEY (id) REFERENCES nodes(id)\
)")
conn.commit()
nodetag_df=pd.read_csv('nodes_tags.csv')
nodetag_df.to_sql('nodes_tags', conn, if_exists='append', index=False)

cur.execute("CREATE TABLE ways (\
    id INTEGER PRIMARY KEY NOT NULL,\
    user TEXT,\
    uid INTEGER,\
    version TEXT,\
    changeset INTEGER,\
    timestamp TEXT\
)")
conn.commit()
way_df=pd.read_csv('ways.csv')
way_df.to_sql('ways', conn, if_exists='append', index=False)

cur.execute("CREATE TABLE ways_nodes (\
    id INTEGER NOT NULL,\
    node_id INTEGER NOT NULL, \
    position INTEGER NOT NULL, \
    FOREIGN KEY (id) REFERENCES ways(id),\
    FOREIGN KEY (node_id) REFERENCES nodes(id)\
)")
conn.commit()
waynode_df=pd.read_csv('ways_nodes.csv')
waynode_df.to_sql('ways_nodes', conn, if_exists='append', index=False)


cur.execute("CREATE TABLE ways_tags (\
    id INTEGER NOT NULL,\
    key TEXT NOT NULL,\
    value TEXT NOT NULL,\
    type TEXT,\
    FOREIGN KEY (id) REFERENCES ways(id)\
)")
conn.commit()
waytag_df=pd.read_csv('ways_tags.csv')
waytag_df=waytag_df.dropna(subset=['id', 'key', 'value'], how='any')
waytag_df.to_sql('ways_tags', conn, if_exists='append', index=False)
conn.commit()
conn.close()

In [5]:
directory = os.getcwd()
for root, dirs, files in os.walk(directory, topdown=False):
    for name in files:
        f = os.path.join(root, name)
        print (name, naturalsize(os.path.getsize(f)))

('osmProject-checkpoint.ipynb', '37.0 kB')
('nodes.csv', '988.5 MB')
('nodes_tags.csv', '28.5 MB')
('phoenix.db', '1.5 GB')
('Phoenix.osm', '2.7 GB')
('phoenix2.osm', '0 Bytes')
('phx2.osm', '39.5 MB')
('phx3.osm', '2.7 GB')
('phxmed.osm', '27.4 MB')
('phxsmall.osm', '302.6 kB')
('ways.csv', '101.3 MB')
('ways_nodes.csv', '325.4 MB')
('ways_tags.csv', '240.0 MB')
('nodes.csv', '989.9 MB')
('nodes_tags.csv', '28.6 MB')
('osmProject.ipynb', '23.3 kB')
('phoenix.db', '1.5 GB')
('Phoenix.osm', '2.7 GB')
('phxmed.osm', '27.4 MB')
('phxsmall.osm', '270.4 kB')
('ways.csv', '101.4 MB')
('ways_nodes.csv', '325.8 MB')
('ways_tags.csv', '239.9 MB')


In [6]:
conn=sqlite3.connect('phoenix.db')
conn.text_factory = lambda x: unicode(x, 'utf-8', 'ignore')
cur = conn.cursor() 
query='select count(id) from nodes; '
total_rows=0
result=cur.execute(query)
for row in result:
    print 'Nodes in database:',  row[0]
    total_rows+=row[0]

query='select count(id) from ways;'

result = cur.execute(query)
for row in result:
    print 'Ways in database:',  row[0]
    total_rows+=row[0]
    
query='select count(id) from nodes_tags;'

result = cur.execute(query)
for row in result:
    print 'Nodes Tags in database:',  row[0]
    total_rows+=row[0]
    
query='select count(id) from ways_nodes;'

result = cur.execute(query)
for row in result:
    print 'Ways Nodes in database:',  row[0]
    total_rows+=row[0]
    
query='select count(id) from ways_tags;'

result = cur.execute(query)
for row in result:
    print 'Ways Tags in database:',  row[0]
    total_rows+=row[0]

print '\nTotal Rows in database:\033[1m', total_rows, "\033[0m"

    
query='select count(distinct(user)) from (select user from nodes union all select user from ways);'
result=cur.execute(query)
for row in result:
    print '\nTotal users that contributed: ', row[0]
    
query='select user, count(*) from nodes group by user order by count(*) desc limit 10;'
print '\nThe top 10 contributors:'
for row in cur.execute(query):
    print (row)
    
query='SELECT DISTINCT key, Count(*) AS [Count] FROM nodes_tags GROUP BY nodes_tags.key ORDER BY Count(*) DESC limit 10;'

result=cur.execute(query)
print '\nTop 10 distinct keys:'
for row in result:
    print (row)
    
query="SELECT DISTINCT value, Count(*) AS [Count] FROM nodes_tags GROUP BY value, key HAVING (((key)='brand')) ORDER BY Count(*) DESC limit 10;"

result=cur.execute(query)
print '\nTop 10 brands keys:'
for row in result:
    print (row)
    
query="SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i ON nodes_tags.id=i.id WHERE nodes_tags.key='cuisine' GROUP BY nodes_tags.value ORDER BY num DESC LIMIT 10;"

result=cur.execute(query)
print '\nTop 10 cuisines:'
for row in result:
    print (row)
conn.close()

Nodes in database: 11456335
Ways in database: 1648767
Nodes Tags in database: 763810
Ways Nodes in database: 13518908
Ways Tags in database: 6645069

Total Rows in database: 34032889 

Total users that contributed:  3437

The top 10 contributors:
(u'_jcaruso', 3009230)
(u'Dr Kludge', 1898520)
(u'TheDutchMan13', 664651)
(u'Dr Kludge{import}', 539009)
(u'tomthepom', 233832)
(u'AJ Riley', 204903)
(u'HJUdall', 200963)
(u'Adam Martin', 184816)
(u'adenium', 168604)
(u'MapperMudkip', 152505)

Top 10 distinct keys:
(u'highway', 134122)
(u'natural', 41624)
(u'power', 36642)
(u'barrier', 34827)
(u'amenity', 31600)
(u'direction', 29576)
(u'crossing', 25805)
(u'name', 22110)
(u'street', 18169)
(u'housenumber', 18030)

Top 10 brands keys:
(u'Starbucks', 134)
(u'Subway', 128)
(u'Circle K', 112)
(u'Blink', 62)
(u'Grid', 58)
(u'Wells Fargo', 53)
(u'CVS Pharmacy', 52)
(u'T-Mobile', 51)
(u'Little Caesars', 47)
(u'Walmart', 42)

Top 10 cuisines:
(u'pizza', 160)
(u'mexican', 152)
(u'american', 62)
(u'chin